# Laboratorio 6 

Francis Aguilar  #22243
Gerardo Pineda  #22880
Angela García  #22869

Enlace al repositorio: https://github.com/faguilarleal/laboratorio6_IA


Enlace al video: https://youtu.be/mYy3hG4wFNw

# Task 1  

**1. En un juego de suma cero para dos jugadores, ¿cómo funciona el algoritmo minimax para determinar la estrategia óptima para cada jugador? ¿Puede explicarnos el concepto de "valor minimax" y su importancia en este contexto?**
El funcionamiento del algoritmo consiste en construir un árbol de juego donde cada nodo representa un estado del juego y las ramas corresponden a las posibles jugadas. En los niveles donde le toca jugar al primer jugador (Max), se elige la jugada con el mayor valor posible, mientras que en los niveles donde le toca jugar al oponente (Min), se elige la jugada con el menor valor. Este proceso se repite de manera recursiva hasta llegar al nodo raíz, lo que permite determinar la mejor jugada inicial.

El valor minimax de un nodo es el puntaje que un jugador puede asegurar si ambos juegan de manera óptima a partir de ese estado. Su importancia radica en que permite evaluar qué tan ventajosa o desventajosa es una posición en el juego, guiando a los jugadores hacia decisiones óptimas basadas en la anticipación de los movimientos del oponente.



**2. Compare y contraste el algoritmo minimax con la poda alfa-beta. ¿Cómo mejora la poda alfa-beta la eficiencia del algoritmo minimax, particularmente en árboles de caza grandes? Proporcione un ejemplo para ilustrar la diferencia en la complejidad computacional entre la poda minimax y alfa-beta.**
Para mejorar esta eficiencia, se utiliza la poda alfa-beta, que optimiza minimax descartando ramas del árbol que no influirán en la decisión final. Este algoritmo introduce dos valores, alfa y beta, que representan los mejores valores encontrados para los jugadores Max y Min, respectivamente. Si en algún momento un nodo tiene un valor que garantiza que el oponente nunca lo elegirá, se puede dejar de evaluar esa rama, reduciendo significativamente el número de nodos explorados.


**3. ¿Cuál es el papel de expectiminimax en juegos con incertidumbre, como aquellos que involucran nodos de azar o información oculta? ¿En qué se diferencia el expectiminimax del minimax en el manejo de resultados probabilísticos y cuáles son los desafíos clave que aborda?**

El algoritmo expectiminimax es una extensión de minimax diseñada para juegos en los que hay incertidumbre, como aquellos que incluyen azar o información oculta. Mientras que minimax asume que cada jugador tiene control total sobre sus decisiones, expectiminimax introduce nodos de azar, que representan eventos aleatorios como el lanzamiento de un dado o la repartición de cartas.

El funcionamiento del algoritmo es similar al minimax, con la diferencia de que, además de los nodos Max y Min, se incluyen nodos de azar. En estos nodos, en lugar de elegir el mejor o peor resultado, se calcula un valor esperado basado en las probabilidades de cada posible desenlace. Esto permite modelar situaciones en las que las decisiones no son completamente deterministas, sino que dependen de factores probabilísticos.

## Connect Four

### Tablero

In [1]:
import numpy as np
import random
import math
from IPython.display import clear_output


In [2]:
class Conecta4:
    FILAS = 6
    COLUMNAS = 7
    JUGADOR = 1
    IA = 2

    def __init__(self):
        self.tablero = np.zeros((self.FILAS, self.COLUMNAS), dtype=int)  

    def imprimir_tablero(self):
        print(np.flip(self.tablero, 0)) 

    def es_movimiento_valido(self, columna):
        return self.tablero[self.FILAS - 1, columna] == 0  

    def obtener_fila_disponible(self, columna):
        for fila in range(self.FILAS):
            if self.tablero[fila, columna] == 0:
                return fila
        return None  

    def realizar_movimiento(self, columna, jugador):
        if not self.es_movimiento_valido(columna):
            return False  
        fila = self.obtener_fila_disponible(columna)
        self.tablero[fila, columna] = jugador
        return True

    def verificar_victoria(self, jugador):
        for fila in range(self.FILAS):
            for col in range(self.COLUMNAS - 3):
                if all(self.tablero[fila, col + i] == jugador for i in range(4)):
                    return True

        for col in range(self.COLUMNAS):
            for fila in range(self.FILAS - 3):
                if all(self.tablero[fila + i, col] == jugador for i in range(4)):
                    return True

        for fila in range(self.FILAS - 3):
            for col in range(self.COLUMNAS - 3):
                if all(self.tablero[fila + i, col + i] == jugador for i in range(4)):
                    return True

        for fila in range(3, self.FILAS):
            for col in range(self.COLUMNAS - 3):
                if all(self.tablero[fila - i, col + i] == jugador for i in range(4)):
                    return True

        return False  

    def tablero_lleno(self):
        return np.all(self.tablero != 0)

    def obtener_movimientos_validos(self):
        return [c for c in range(self.COLUMNAS) if self.es_movimiento_valido(c)]

    def clonar_tablero(self):
        nuevo = Conecta4()
        nuevo.tablero = np.copy(self.tablero)
        return nuevo

# ====================== PRUEBA ======================
if __name__ == "__main__":
    tablero = Conecta4()
    tablero.imprimir_tablero()

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]


Creacion del tablero usando IA Generativa: Prompt utilzado: "Tengo que hacer un conecta 4 con un agente en python, podrias hacerme el tablero y la validacion de las reglas, esto es con el objetivo de poder aplicar minimax"

![image](./img/image.png)

## Agente

In [23]:
class AgenteIA:
    def __init__(self, profundidad=4, poda_alpha_beta=True):
        self.profundidad = profundidad
        self.poda_alpha_beta = poda_alpha_beta

    def evaluar_ventana(self, ventana, jugador):
        puntuacion = 0
        oponente = 3 - jugador  

        # 4 en línea → Victoria
        if np.count_nonzero(ventana == jugador) == 4:  
            puntuacion += 1000  
        elif np.count_nonzero(ventana == jugador) == 3 and np.count_nonzero(ventana == 0) == 1:
            puntuacion += 100  # Muy fuerte
        elif np.count_nonzero(ventana == jugador) == 2 and np.count_nonzero(ventana == 0) == 2:
            puntuacion += 10  # Buena jugada
        elif np.count_nonzero(ventana == oponente) == 3 and np.count_nonzero(ventana == 0) == 1:
            puntuacion -= 100  # Bloquear jugada peligrosa
        elif np.count_nonzero(ventana == oponente) == 2 and np.count_nonzero(ventana == 0) == 2:
            puntuacion -= 10  # Bloqueo medio

        return puntuacion


    def evaluar_posicion(self, tablero, jugador):
        if tablero.verificar_victoria(jugador):
            return 1000
        elif tablero.verificar_victoria(3 - jugador):  
            return -1000
        else:
            return random.randint(-10, 10)  

        # # Esto de aca abajo es mas complejo y mejor pero tarda mucho en ejecutarse
        # if tablero.verificar_victoria(jugador):
        #     return 1000
        # elif tablero.verificar_victoria(3 - jugador):
        #     return -1000
        
        # puntuacion = 0

        # # Ponderación para fichas en la columna central (son más fuertes)
        # columna_central = [fila[tablero.COLUMNAS // 2] for fila in tablero.tablero]
        # puntuacion += columna_central.count(jugador) * 3  

        
        # for fila in range(tablero.FILAS):
        #     for col in range(tablero.COLUMNAS - 3):  # Horizontal
        #         ventana = tablero.tablero[fila, col:col + 4]
        #         puntuacion += self.evaluar_ventana(ventana, jugador)

        # for col in range(tablero.COLUMNAS):
        #     for fila in range(tablero.FILAS - 3):  # Vertical
        #         ventana = tablero.tablero[fila:fila+4, col]
        #         puntuacion += self.evaluar_ventana(ventana, jugador)

        # for fila in range(tablero.FILAS - 3):
        #     for col in range(tablero.COLUMNAS - 3):  # Diagonal ↘
        #         ventana = [tablero.tablero[fila+i, col+i] for i in range(4)]
        #         puntuacion += self.evaluar_ventana(ventana, jugador)

        # for fila in range(3, tablero.FILAS):
        #     for col in range(tablero.COLUMNAS - 3):  # Diagonal ↙
        #         ventana = [tablero.tablero[fila-i, col+i] for i in range(4)]
        #         puntuacion += self.evaluar_ventana(ventana, jugador)

        # return puntuacion

    def minimax(self, tablero, profundidad, alpha, beta, maximizando):
        if profundidad == 0 or tablero.verificar_victoria(Conecta4.JUGADOR) or tablero.verificar_victoria(Conecta4.IA) or tablero.tablero_lleno():
            return self.evaluar_posicion(tablero, Conecta4.IA)

        movimientos = tablero.obtener_movimientos_validos()

        if maximizando:  
            max_eval = -np.inf
            for columna in movimientos:
                tablero_copia = tablero.clonar_tablero()
                tablero_copia.realizar_movimiento(columna, Conecta4.IA)
                evaluacion = self.minimax(tablero_copia, profundidad - 1, alpha, beta, False)
                max_eval = max(max_eval, evaluacion)

                if self.poda_alpha_beta:
                    alpha = max(alpha, evaluacion)
                    if beta <= alpha:
                        break  
            return max_eval
        else:  
            min_eval = np.inf
            for columna in movimientos:
                tablero_copia = tablero.clonar_tablero()
                tablero_copia.realizar_movimiento(columna, Conecta4.JUGADOR)
                evaluacion = self.minimax(tablero_copia, profundidad - 1, alpha, beta, True)
                min_eval = min(min_eval, evaluacion)

                if self.poda_alpha_beta:
                    beta = min(beta, evaluacion)
                    if beta <= alpha:
                        break  
            return min_eval

    def mejor_movimiento(self, tablero):
        """ Calcula el mejor movimiento disponible usando Minimax """
        mejor_columna = random.choice(tablero.obtener_movimientos_validos())  
        mejor_valor = -np.inf
        alpha, beta = -np.inf, np.inf

        for columna in tablero.obtener_movimientos_validos():
            tablero_copia = tablero.clonar_tablero()
            tablero_copia.realizar_movimiento(columna, Conecta4.IA)
            valor_movimiento = self.minimax(tablero_copia, self.profundidad, alpha, beta, False)

            if valor_movimiento > mejor_valor:
                mejor_valor = valor_movimiento
                mejor_columna = columna

        return mejor_columna

## Humano vs IA

In [18]:
def play_IA(alpha_beta=True): 
    juego = Conecta4()
    agente = AgenteIA(profundidad=4, poda_alpha_beta=alpha_beta)

    turno_ia = False  

    while not juego.tablero_lleno():
        clear_output(wait=False)
        juego.imprimir_tablero()

        if juego.verificar_victoria(Conecta4.JUGADOR):
            print("¡Ganaste!")
            break
        elif juego.verificar_victoria(Conecta4.IA):
            print("La IA ha ganado.")
            break

        if turno_ia:
            print("Turno de la IA...")
            mejor_columna = agente.mejor_movimiento(juego)
            if juego.realizar_movimiento(mejor_columna, Conecta4.IA):
                turno_ia = not turno_ia
        else:
            print("Tu turno:")
            juego.imprimir_tablero()
            columna = int(input("Selecciona una columna (0-6): "))
            if juego.realizar_movimiento(columna, Conecta4.JUGADOR):
                turno_ia = not turno_ia  

    juego.imprimir_tablero()
    print("Fin del juego")

# IA vs IA

In [19]:
def IA_vs_IA(alpha_beta1 = True, alpha_beta2 = True):
    juego = Conecta4()
    agente1 = AgenteIA(profundidad=4, poda_alpha_beta=alpha_beta1)
    agente2 = AgenteIA(profundidad=4, poda_alpha_beta=alpha_beta2)

    turno_ia = True  

    while not juego.tablero_lleno():
        clear_output(wait=False)
        juego.imprimir_tablero()

        if juego.verificar_victoria(Conecta4.JUGADOR):
            print("¡IA 1 ganó!")
            break
        elif juego.verificar_victoria(Conecta4.IA):
            print("¡IA 2 ganó!")
            break

        if turno_ia:
            print("Turno de la IA 1...")
            mejor_columna = agente1.mejor_movimiento(juego)
            juego.realizar_movimiento(mejor_columna, Conecta4.JUGADOR)
        else:
            print("Turno de la IA 2...")
            mejor_columna = agente2.mejor_movimiento(juego)
            juego.realizar_movimiento(mejor_columna, Conecta4.IA)

        turno_ia = not turno_ia  

    juego.imprimir_tablero()
    print("Fin del juego")

## JUGAR

In [33]:
if __name__ == "__main__":
    print("Seleccione el modo de juego que desea jugar:\n1. Jugador Humano vs IA\n2. IA vs IA\n")
    opcion = input("Ingrese 1 o 2: ")
    if opcion == "1":
        res = input("¿Quieres que el agente use la tecnica de alpha-beta pruning? (y/n): ").strip().lower()
        play_IA(res == "y")
    elif opcion == "2":
        res = input("¿Quieres que el agente 1 use la tecnica de alpha-beta pruning? (y/n): ").strip().lower()
        res2 = input("¿Quieres que el agente 2 use la tecnica de alpha-beta pruning? (y/n): ").strip().lower()
        IA_vs_IA(res == "y", res2 == "y")
    else:
        print("Opción no válida. Reinicie el juego.")


[[1 2 1 0 0 0 0]
 [2 1 1 0 0 0 0]
 [2 1 2 0 0 0 0]
 [1 2 1 2 2 2 2]
 [2 1 1 2 1 2 1]
 [1 2 1 2 1 1 2]]
¡IA 2 ganó!
[[1 2 1 0 0 0 0]
 [2 1 1 0 0 0 0]
 [2 1 2 0 0 0 0]
 [1 2 1 2 2 2 2]
 [2 1 1 2 1 2 1]
 [1 2 1 2 1 1 2]]
Fin del juego
